In [1]:
import pandas as pd
import numpy as np
import datetime
import logging

from csh_fantasy_bot import bot
from csh_fantasy_bot.roster_change_optimizer import RosterChangeSet
from csh_fantasy_bot.nhl import score_team



In [2]:
# true value ignores actuals if they exist and works off projections only
simulation_mode=False
week_number = 5
league_id = '403.l.41177'
# league_id = "403.l.18782"
manager = bot.ManagerBot(week_number, league_id=league_id, simulation_mode=simulation_mode)
league_scorer = manager.score_comparer
manager.opp_team_name

"Kevan's Best Team"

In [8]:
# paste roster change text into here
"""
Here is example of what roster change text should look like
It is output from the rosterchangeset.pretty_print method

Date: 2021-02-08, in: Alexis Lafreniere(8641), out: Teuvo Teravainen(5698)
Date: 2021-02-09, in: Zdeno Chara(1700), out: Kailer Yamamoto(7537)
Date: 2021-02-10, in: Adam Fox(7174), out: Evander Kane(4684)
"""
roster_change_text="""
Date: 2021-02-15, in: Anthony Cirelli(6814), out: Samuel Girard(7155)
Date: 2021-02-17, in: Andrew Copp(6083), out: Nazem Kadri(4687)
"""

roster_changes = RosterChangeSet.from_pretty_print_text(roster_change_text, manager.all_player_predictions)
scores_with_changes = manager.score_team(roster_change_set=roster_changes)[1]
scoring_result = league_scorer.score(scores_with_changes)[manager.stat_categories]
print(scoring_result)
print(f"\nScore: opp->({scoring_result.loc['score_opp'].sum()}), league->({scoring_result.loc['score_league'].sum()})") 
if (roster_change_text != ''):
    scores_ignore_roster_changes = manager.score_team()[1]
    score_result_no_changes = league_scorer.score(scores_ignore_roster_changes)
    print (f"\nWithout roster changes: opp->({score_result_no_changes.loc['score_opp'].sum()}), league->({score_result_no_changes.loc['score_league'].sum()})") 

G       A    +/-     PIM      SOG       FW     HIT
my-scores          15.310  21.940  3.730  39.010  142.880  147.850  93.190
opponent           12.870  24.850  5.620  22.190  125.070   97.350  48.990
difference_opp      2.440  -2.910 -1.890  16.820   17.810   50.500  44.200
mean-league        12.551  21.355  3.101  22.907  111.132  112.221  49.454
std dev             2.020   3.186  1.662   6.343   20.633   35.813  16.555
mean-opp           14.090  23.395  4.675  30.600  133.975  122.600  71.090
num_stds_opp        1.208  -0.913 -1.137   2.652    0.863    1.410   2.670
score_opp           1.208  -0.913 -1.137   1.500    0.863    1.410   1.500
win_loss_opp        1.000  -1.000 -1.000   1.000    1.000    1.000   1.000
difference_league   2.759   0.585  0.629  16.103   31.748   35.629  43.736
score_league        1.366   0.184  0.379   1.500    1.500    0.995   1.500

Score: opp->(4.431), league->(7.424)

Without roster changes: opp->(1.641), league->(3.73)


In [7]:
scores_ignore_roster_changes = scores_with_changes
games_played = scores_ignore_roster_changes['G'].groupby(level=1).count()
result = manager.all_player_predictions.copy()
result['GP'] = games_played
result[result.GP.notna()].sort_values('GP', ascending=False).round(2)[['name', 'GP', 'fpts']]

,name,GP,fpts
player_id,,,
5696,Tom Wilson,4.0,1.06
4990,Brock Nelson,4.0,0.83
5986,Darnell Nurse,4.0,0.65
5985,Sean Monahan,4.0,0.56
5370,Dougie Hamilton,4.0,0.79
5431,Vincent Trocheck,4.0,0.48
7902,Brady Tkachuk,4.0,1.13
3358,Brent Burns,3.0,0.78
4684,Evander Kane,3.0,1.77
